In [3]:
import pandas as pd
import numpy as np
from pandas.core.common import random_state
from pandas.core.interchange.from_dataframe import categorical_column_to_series
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import plot_importance
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score
import shap
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [3]:
chunk_size = 100000

In [5]:
# Initialize variables for analysis
total_rows = 0
column_sums = None
column_squared_sums = None

In [6]:
# Step 3: Read the data in chunks
def read_csv_in_chunks(file_path, chunk_size):
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        yield chunk

In [8]:
# Step 4: Sample 20% of observation
labels = pd.read_csv('data/train_labels.csv')
sample_labels = labels.sample(frac=0.2, random_state=42)

In [10]:
# Read data
sampled_data = []
for chunk in read_csv_in_chunks('data/train_data.csv', chunk_size):
    merged_chunk = pd.merge(chunk, sample_labels, on='customer_ID', how='inner')
    sampled_data.append(merged_chunk)
    
# Combine all chunks into a single dataframe
development_sample = pd.concat(sampled_data, ignore_index=True)

# Save the development sample
development_sample.to_csv('data/development_sample.csv', index=False)

In [8]:
# load development_sample
data = []
for chunk in read_csv_in_chunks('data/development_sample.csv', chunk_size):
    data.append(chunk)

    

In [13]:
df = pd.concat(data, ignore_index=True)
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_137,D_138,D_139,D_140,D_141,D_142,D_143,D_144,D_145,target
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-03-11,0.374606,0.033519,0.044293,1.008622,0.001470,0.459235,0.002339,0.006168,...,NaN,NaN,0.008263,0.006609,0.007370,NaN,0.007171,0.005120,0.007513,0
1,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-04-22,0.414269,0.002516,0.059667,0.123964,0.004374,0.434148,0.001405,0.052130,...,NaN,NaN,0.001986,0.004050,0.000796,NaN,0.001802,0.002364,0.003987,0
2,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-05-12,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,...,NaN,NaN,0.009515,0.008757,0.009219,NaN,0.003134,0.001686,0.001265,0
3,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-06-10,0.328983,0.038574,0.049463,0.115654,0.004654,0.416112,0.003223,0.081001,...,NaN,NaN,0.002524,0.007841,0.007421,NaN,0.000728,0.003591,0.007998,0
4,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-07-19,0.496989,0.005552,0.041452,0.133631,0.007363,0.419864,0.003393,0.098308,...,NaN,NaN,0.003823,0.009599,0.006957,NaN,0.008746,0.007101,0.006658,0


In [37]:
categorical_columns = ['B_30', 'B_38', 'D_114', 'D_116', 'D_117', 'D_120', 'D_126', 'D_63', 'D_64', 'D_66', 'D_68']

In [26]:
# Step 5: One-hot encoding for categorical variables
# List of known categorical columns
# List of categorical columns

# Function to read and process data in chunks
def process_chunks(file_path, chunk_size=100000):
    encoded_chunks = []
    
    for chunk in pd.read_csv(file_path, chunksize=chunk_size):
        # Convert all specified columns to strings
        for col in categorical_columns:
            chunk[col] = chunk[col].astype(str)
        
        # Perform one-hot encoding
        encoded_chunk = pd.get_dummies(chunk, columns=categorical_columns)
        
        # Ensure the encoded columns are of type int (0 or 1)
        for col in encoded_chunk.columns:
            if col.startswith(tuple(categorical_columns)):
                encoded_chunk[col] = encoded_chunk[col].astype(int)
        
        encoded_chunks.append(encoded_chunk)
        
        print(f"Processed chunk of size {len(chunk)}")
    
    return pd.concat(encoded_chunks, ignore_index=True)

# Process the file
df_encoded = process_chunks('data/development_sample.csv')

print("One-hot encoding completed.")
print(f"Shape of encoded DataFrame: {df_encoded.shape}")

# Display the first few rows of the encoded DataFrame
print(df_encoded.head())

# Check the data types of the encoded columns
print(df_encoded.dtypes)

Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 100000
Processed chunk of size 7082
One-hot encoding completed.
Shape of encoded DataFrame: (1107082, 235)
                                         customer_ID         S_2       P_2  \
0  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-03-11  0.374606   
1  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-04-22  0.414269   
2  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-05-12  0.413310   
3  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-06-10  0.328983   
4  000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...  2017-07-19  0.496989   

       D_39       B_1       B_2       R_1       S_3      D_41       B_3  ...  \
0  0.03351

In [27]:
df_encoded.to_csv('data/train_encoded_data.csv', index=False)

***USE THE BELOW CODE FOR QUESTION 6~***

In [26]:
# Step 6: Feature Engineering
df = pd.read_csv('data/train_encoded_data.csv')


In [28]:
target_df = df[['customer_ID', 'target']].groupby('customer_ID').sum()

In [29]:
target_df.head()

,target
customer_ID,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,0
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,0
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,0


In [31]:
df['S_2'] = pd.to_datetime(df['S_2'])
df = df.drop(columns=['target'])
df.head()

,customer_ID,S_2,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,...,D_66_1.0,D_66_nan,D_68_0.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0,D_68_nan
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-03-11,0.374606,0.033519,0.044293,1.008622,0.001470,0.459235,0.002339,0.006168,...,0,1,0,0,0,0,0,0,0,1
1,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-04-22,0.414269,0.002516,0.059667,0.123964,0.004374,0.434148,0.001405,0.052130,...,0,1,0,0,0,0,0,0,0,1
2,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-05-12,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,...,0,1,0,0,1,0,0,0,0,0
3,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-06-10,0.328983,0.038574,0.049463,0.115654,0.004654,0.416112,0.003223,0.081001,...,0,1,0,0,1,0,0,0,0,0
4,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,2017-07-19,0.496989,0.005552,0.041452,0.133631,0.007363,0.419864,0.003393,0.098308,...,0,1,0,0,1,0,0,0,0,0


In [32]:
# Numerical Columns:
numerical_columns = df.select_dtypes(include=[np.number]).columns.tolist()

In [33]:
df[numerical_columns].head()

,P_2,D_39,B_1,B_2,R_1,S_3,D_41,B_3,D_42,D_43,...,D_66_1.0,D_66_nan,D_68_0.0,D_68_1.0,D_68_2.0,D_68_3.0,D_68_4.0,D_68_5.0,D_68_6.0,D_68_nan
0,0.374606,0.033519,0.044293,1.008622,0.001470,0.459235,0.002339,0.006168,0.152932,NaN,...,0,1,0,0,0,0,0,0,0,1
1,0.414269,0.002516,0.059667,0.123964,0.004374,0.434148,0.001405,0.052130,0.154147,0.053474,...,0,1,0,0,0,0,0,0,0,1
2,0.413310,0.003285,0.053418,0.304955,0.002316,0.415906,0.009388,0.048780,0.150513,0.035667,...,0,1,0,0,1,0,0,0,0,0
3,0.328983,0.038574,0.049463,0.115654,0.004654,0.416112,0.003223,0.081001,0.148605,0.166655,...,0,1,0,0,1,0,0,0,0,0
4,0.496989,0.005552,0.041452,0.133631,0.007363,0.419864,0.003393,0.098308,0.147616,0.117810,...,0,1,0,0,1,0,0,0,0,0


In [34]:
# Aggregation For all the numerical columns

def calculate_numerical_statistics(df, time_period):
    last_date = df['S_2'].max()
    time_period_ago = last_date - pd.DateOffset(months=time_period)

    data_last_time_period = df[(df['S_2'] > time_period_ago) & (df['S_2'] <= last_date)]

    average_spend_last_time_period = data_last_time_period.groupby('customer_ID')[numerical_columns].agg(['mean', 'min', 'max', 'sum'])
    average_spend_last_time_period.columns = [f'{col}_{agg}_{time_period}' for col, agg in average_spend_last_time_period.columns]

    return average_spend_last_time_period


# Calculate statistics for different time periods
average_spend_3_months = calculate_numerical_statistics(df, 3)
average_spend_6_months = calculate_numerical_statistics(df, 6)
average_spend_9_months = calculate_numerical_statistics(df, 9)
average_spend_12_months = calculate_numerical_statistics(df, 12)

feature_engineered_df = pd.merge(average_spend_3_months, average_spend_6_months, on='customer_ID', how='inner')
feature_engineered_df = pd.merge(feature_engineered_df, average_spend_9_months, on='customer_ID', how='inner')
feature_engineered_df = pd.merge(feature_engineered_df, average_spend_12_months, on='customer_ID', how='inner')

feature_engineered_df.head()


,P_2_mean_3,P_2_min_3,P_2_max_3,P_2_sum_3,D_39_mean_3,D_39_min_3,D_39_max_3,D_39_sum_3,B_1_mean_3,B_1_min_3,...,D_68_5.0_max_12,D_68_5.0_sum_12,D_68_6.0_mean_12,D_68_6.0_min_12,D_68_6.0_max_12,D_68_6.0_sum_12,D_68_nan_mean_12,D_68_nan_min_12,D_68_nan_max_12,D_68_nan_sum_12
customer_ID,,,,,,,,,,,,,,,,,,,,,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0.447801,0.414444,0.477116,1.343404,0.014288,0.000467,0.035885,0.042865,0.069795,0.009413,...,0,0,0.000000,0,0,0,0.083333,0,1,1
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,0.976846,0.974383,0.978897,2.930539,0.002024,0.001221,0.002629,0.006072,0.008831,0.006584,...,0,0,1.000000,1,1,12,0.000000,0,0,0
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,0.638958,0.634894,0.642295,1.916874,0.574667,0.415254,0.685210,1.724002,0.431032,0.429032,...,1,12,0.000000,0,0,0,0.000000,0,0,0
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,0.973429,0.963991,0.980221,2.920287,0.268469,0.009431,0.470704,0.805407,0.023506,0.019392,...,1,7,0.416667,0,1,5,0.000000,0,0,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,0.993352,0.968914,1.006598,2.980056,0.092158,0.005624,0.150871,0.276473,0.040335,0.026597,...,0,0,1.000000,1,1,12,0.000000,0,0,0


In [35]:
feature_engineered_df.to_csv('data/feature_engineered_data.csv')

In [38]:
# Categorical Columns:
one_hot_encoded_col = []
for ftr in df.columns:
    for column in categorical_columns:
        if ftr.startswith(column):
            one_hot_encoded_col.append(ftr)

In [39]:
# feature engineering for categorical values 
def calculate_categorical_statistics(df, time_period, one_hot_encoded_cols):
    # Get the last date in the dataset
    last_date = df['S_2'].max()
    
    # Calculate the date for the start of the time period
    time_period_start = last_date - pd.DateOffset(months=time_period)
    
    # Filter the data for the specified time period
    data_last_time_period = df[(df['S_2'] > time_period_start) & (df['S_2'] <= last_date)]
    
    # Calculate the positive response rates for each customer
    positive_response_rates = data_last_time_period.groupby('customer_ID')[one_hot_encoded_cols].mean()
    positive_response_rates.columns = [f'{col}_response_rate_{time_period}' for col in positive_response_rates.columns]
    
    # Calculate whether each customer ever responded positively
    ever_responded = data_last_time_period.groupby('customer_ID')[one_hot_encoded_cols].any().astype(int)
    ever_responded.columns = [f'{col}_ever_respond_{time_period}' for col in ever_responded.columns]
    
    # Merge the two DataFrames on 'customer_ID'
    merged_df = pd.merge(positive_response_rates, ever_responded, on='customer_ID', how='inner')
    
    return merged_df

cat_stats_3_months = calculate_categorical_statistics(df, 3, one_hot_encoded_col)
cat_stats_6_months = calculate_categorical_statistics(df, 6, one_hot_encoded_col)
cat_stats_9_months = calculate_categorical_statistics(df, 9, one_hot_encoded_col)
cat_stats_12_months = calculate_categorical_statistics(df, 12, one_hot_encoded_col)

final_categorical_df = pd.merge(cat_stats_3_months, cat_stats_6_months, on='customer_ID', how='inner')
final_categorical_df = pd.merge(final_categorical_df, cat_stats_9_months, on='customer_ID', how='inner')
final_feature_engineered_df = pd.merge(final_categorical_df, cat_stats_12_months, on='customer_ID', how='inner')

final_feature_engineered_df.head()

,B_30_0.0_response_rate_3,B_30_1.0_response_rate_3,B_30_2.0_response_rate_3,B_30_nan_response_rate_3,B_38_1.0_response_rate_3,B_38_2.0_response_rate_3,B_38_3.0_response_rate_3,B_38_4.0_response_rate_3,B_38_5.0_response_rate_3,B_38_6.0_response_rate_3,...,D_66_1.0_ever_respond_12,D_66_nan_ever_respond_12,D_68_0.0_ever_respond_12,D_68_1.0_ever_respond_12,D_68_2.0_ever_respond_12,D_68_3.0_ever_respond_12,D_68_4.0_ever_respond_12,D_68_5.0_ever_respond_12,D_68_6.0_ever_respond_12,D_68_nan_ever_respond_12
customer_ID,,,,,,,,,,,,,,,,,,,,,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0.666667,0.333333,0.0,0.0,0.0,0.0,0.333333,0.0,0.666667,0.0,...,0,1,0,0,1,1,0,0,0,1
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,1.000000,0.000000,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.0,...,0,1,0,0,0,0,0,0,1,0
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,0.666667,0.333333,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0,1,0,0,0,0,0,1,0,0
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,1.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,...,0,1,0,0,0,0,0,1,1,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,1.000000,0.000000,0.0,0.0,0.0,1.0,0.000000,0.0,0.000000,0.0,...,0,1,0,0,0,0,0,0,1,0


In [40]:
# Integrate feature engineered data
feature_engineered_df = pd.merge(feature_engineered_df, final_feature_engineered_df, on='customer_ID', how='inner')
feature_engineered_df.head()

,P_2_mean_3,P_2_min_3,P_2_max_3,P_2_sum_3,D_39_mean_3,D_39_min_3,D_39_max_3,D_39_sum_3,B_1_mean_3,B_1_min_3,...,D_66_1.0_ever_respond_12,D_66_nan_ever_respond_12,D_68_0.0_ever_respond_12,D_68_1.0_ever_respond_12,D_68_2.0_ever_respond_12,D_68_3.0_ever_respond_12,D_68_4.0_ever_respond_12,D_68_5.0_ever_respond_12,D_68_6.0_ever_respond_12,D_68_nan_ever_respond_12
customer_ID,,,,,,,,,,,,,,,,,,,,,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0.447801,0.414444,0.477116,1.343404,0.014288,0.000467,0.035885,0.042865,0.069795,0.009413,...,0,1,0,0,1,1,0,0,0,1
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,0.976846,0.974383,0.978897,2.930539,0.002024,0.001221,0.002629,0.006072,0.008831,0.006584,...,0,1,0,0,0,0,0,0,1,0
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,0.638958,0.634894,0.642295,1.916874,0.574667,0.415254,0.685210,1.724002,0.431032,0.429032,...,0,1,0,0,0,0,0,1,0,0
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,0.973429,0.963991,0.980221,2.920287,0.268469,0.009431,0.470704,0.805407,0.023506,0.019392,...,0,1,0,0,0,0,0,1,1,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,0.993352,0.968914,1.006598,2.980056,0.092158,0.005624,0.150871,0.276473,0.040335,0.026597,...,0,1,0,0,0,0,0,0,1,0


In [4]:
feature_engineered_df = pd.read_csv('data/feature_engineered_data.csv')

KeyboardInterrupt: 

In [9]:
feature_engineered_df.head()

,customer_ID,P_2_mean_3,P_2_min_3,P_2_max_3,P_2_sum_3,D_39_mean_3,D_39_min_3,D_39_max_3,D_39_sum_3,B_1_mean_3,...,D_68_5.0_max_12,D_68_5.0_sum_12,D_68_6.0_mean_12,D_68_6.0_min_12,D_68_6.0_max_12,D_68_6.0_sum_12,D_68_nan_mean_12,D_68_nan_min_12,D_68_nan_max_12,D_68_nan_sum_12
0,000098081fde4fd64bc4d503a5d6f86a0aedc425c96f52...,0.447801,0.414444,0.477116,1.343404,0.014288,0.000467,0.035885,0.042865,0.069795,...,0,0,0.000000,0,0,0,0.083333,0,1,1
1,000445609ff2a39d2dd02484899affa5696210a95f6869...,0.976846,0.974383,0.978897,2.930539,0.002024,0.001221,0.002629,0.006072,0.008831,...,0,0,1.000000,1,1,12,0.000000,0,0,0
2,0004837f0c785928a29a6f83f70f4a1c54caec483a773f...,0.638958,0.634894,0.642295,1.916874,0.574667,0.415254,0.685210,1.724002,0.431032,...,1,12,0.000000,0,0,0,0.000000,0,0,0
3,0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704...,0.973429,0.963991,0.980221,2.920287,0.268469,0.009431,0.470704,0.805407,0.023506,...,1,7,0.416667,0,1,5,0.000000,0,0,0
4,00050d84c6d26e26cd2b18c3eed83d3130c270e2361470...,0.993352,0.968914,1.006598,2.980056,0.092158,0.005624,0.150871,0.276473,0.040335,...,0,0,1.000000,1,1,12,0.000000,0,0,0


In [42]:
feature_engineered_df = pd.merge(feature_engineered_df, target_df, on='customer_ID', how='inner')
feature_engineered_df.head()

,P_2_mean_3,P_2_min_3,P_2_max_3,P_2_sum_3,D_39_mean_3,D_39_min_3,D_39_max_3,D_39_sum_3,B_1_mean_3,B_1_min_3,...,D_66_nan_ever_respond_12,D_68_0.0_ever_respond_12,D_68_1.0_ever_respond_12,D_68_2.0_ever_respond_12,D_68_3.0_ever_respond_12,D_68_4.0_ever_respond_12,D_68_5.0_ever_respond_12,D_68_6.0_ever_respond_12,D_68_nan_ever_respond_12,target
customer_ID,,,,,,,,,,,,,,,,,,,,,
000098081fde4fd64bc4d503a5d6f86a0aedc425c96f5235f98b0f47c9d7d8d4,0.447801,0.414444,0.477116,1.343404,0.014288,0.000467,0.035885,0.042865,0.069795,0.009413,...,1,0,0,1,1,0,0,0,1,0
000445609ff2a39d2dd02484899affa5696210a95f6869f26390bd26eeb3b651,0.976846,0.974383,0.978897,2.930539,0.002024,0.001221,0.002629,0.006072,0.008831,0.006584,...,1,0,0,0,0,0,0,1,0,0
0004837f0c785928a29a6f83f70f4a1c54caec483a773ff4b5b317ac251abda0,0.638958,0.634894,0.642295,1.916874,0.574667,0.415254,0.685210,1.724002,0.431032,0.429032,...,1,0,0,0,0,0,1,0,0,0
0004ec03ca1ab2adb9aa260c61ba5dce8185e19d3ab704029f989240c733b6d0,0.973429,0.963991,0.980221,2.920287,0.268469,0.009431,0.470704,0.805407,0.023506,0.019392,...,1,0,0,0,0,0,1,1,0,0
00050d84c6d26e26cd2b18c3eed83d3130c270e2361470ff272f9409103d067f,0.993352,0.968914,1.006598,2.980056,0.092158,0.005624,0.150871,0.276473,0.040335,0.026597,...,1,0,0,0,0,0,0,1,0,0


In [43]:
feature_engineered_df.to_csv('data/feature_engineered_data.csv')

***USE THE BELOW CODE FOR QUESTION 7~***

In [5]:
df = pd.read_csv('data/feature_engineered_data.csv')

In [6]:
# Step 7; Train-test split
# Use a random seed of 42 for the split.
# Split the data into features and target
X = df.drop(columns=['customer_ID', 'target'])
y = df['target']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
X_test1, X_test2, y_test1, y_test2 = train_test_split(X_test, y_test, test_size=0.5, random_state=42)



In [7]:
# save train and test data as csv
X_train.to_csv('data/X_train.csv', index=False)
y_train.to_csv('data/y_train.csv', index=False)
X_test1.to_csv('data/X_test1.csv', index=False)
y_test1.to_csv('data/y_test1.csv', index=False)
X_test2.to_csv('data/X_test2.csv', index=False)
y_test2.to_csv('data/y_test2.csv', index=False)

In [ ]:
# Step 8: Calculate feature importance using XGBoost

# Load the training data
X_train = pd.read_csv('data/X_train.csv')
y_train = pd.read_csv('data/y_train.csv')

# Initialize the XGBClassifier
xgb_model = xgb.XGBClassifier()

# Fit the model
xgb_model.fit(X_train, y_train)

# # Plot the feature importance
# plot_importance(xgb_model)

# Save the feature importance
feature_importance = xgb_model.feature_importances_
np.save('feature_importance.npy', feature_importance)

# Display the feature importance
print("Feature importance:")
print(feature_importance)

In [19]:
feature_importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
})
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)

In [20]:
feature_importance_df.to_csv('data/feature_importance.csv', index=False)

In [21]:
important_features = feature_importance_df[feature_importance_df['Importance'] > 0.005]
important_features

,Feature,Importance
709,target_mean_3,0.362621
2708,D_126_nan_sum_9,0.123411
3508,target_sum_12,0.056212
1776,D_126_nan_sum_6,0.050182
1704,D_114_nan_sum_6,0.023017
712,target_sum_3,0.021154
3637,D_126_nan_mean_12,0.016584
1644,target_sum_6,0.014891
1913,D_45_mean_9,0.014882
2576,target_sum_9,0.013070


In [22]:
# Apply only importamt features
# Run default XBG
X_train_important = X_train[important_features['Feature']]

xgb_model.fit(X_train_important, y_train)

feature_importance_important = xgb_model.feature_importances_

feature_importance_important_df = pd.DataFrame({
    'Feature': X_train_important.columns,
    'Importance': feature_importance_important
})
feature_importance_important_df = feature_importance_df.sort_values(by='Importance', ascending=False)

feature_importance_important_df


,Feature,Importance
709,target_mean_3,0.362621
2708,D_126_nan_sum_9,0.123411
3508,target_sum_12,0.056212
1776,D_126_nan_sum_6,0.050182
1704,D_114_nan_sum_6,0.023017
...,...,...
2659,D_117_2.0_max_9,0.000000
2658,D_117_2.0_min_9,0.000000
2657,D_117_2.0_mean_9,0.000000
2656,D_117_1.0_sum_9,0.000000


In [23]:
feature_importance_important_df = feature_importance_important_df[feature_importance_important_df['Importance'] > 0.005]
feature_importance_important_df.to_csv('data/feature_importance_important.csv', index=False)

In [24]:
# Run XGB applying parameters 300 trees, 0.5 as learning rate, maximum depth of trees is 4, uses 50% of observation to build each tree, uses 50% of features to build each tree, and assigns a weight of 5 to default observations. 
# Save the model as 'xgb_model_important_features'
# Initialize the XGBClassifier
xgb_model_important_features = xgb.XGBClassifier(
    objective='binary:logistic',
    max_depth=4,
    learning_rate=0.5,
    n_estimators=300,
    subsample=0.5,
    colsample_bytree=0.5,
    reg_alpha=0.1,
    reg_lambda=0.1,
    random_state=42
)

# Apply only important features calculated by default XGB
X_train_featured = X_train_important[feature_importance_important_df['Feature']]

# Create sample weights
sample_weights = np.where(y_train['target'] == 1, 5, 1)

# Fit the model
xgb_model_important_features.fit(X_train_featured, y_train, sample_weight=sample_weights)

# Calculate the feature importance
feature_importance_parameter_model = xgb_model_important_features.feature_importances_



In [25]:
# Save the feature importance
feature_importance_parameter_model_df = pd.DataFrame({
    'Feature': X_train_important.columns,
    'Importance': feature_importance_parameter_model
})
feature_importance_parameter_model_df = feature_importance_parameter_model_df.sort_values(by='Importance', ascending=False)

feature_importance_parameter_model_df = feature_importance_parameter_model_df[feature_importance_parameter_model_df['Importance'] > 0.005]

feature_importance_parameter_model_df.to_csv('data/feature_importance_parameter_model.csv', index=False)

feature_importance_parameter_model_df

,Feature,Importance
0,target_mean_3,0.906530
9,target_sum_9,0.051721
5,target_sum_3,0.011402
6,D_126_nan_mean_12,0.007103
2,target_sum_12,0.006001


In [38]:
### step 9: When to resume training process
# Load the training data, applying important features calculated by step 8
# Load the training data, applying important features calculated by step 8
X_train = pd.read_csv('data/X_train.csv')
feature_importance_parameter_model_df = pd.read_csv('output/feature_importance_parameter_model.csv')
X_train = X_train[feature_importance_parameter_model_df['Feature']]
y_train = pd.read_csv('data/y_train.csv')

X_test1 = pd.read_csv('data/X_test1.csv')
X_test1 = X_test1[feature_importance_parameter_model_df['Feature']]
y_test1 = pd.read_csv('data/y_test1.csv')

X_test2 = pd.read_csv('data/X_test2.csv')
X_test2 = X_test2[feature_importance_parameter_model_df['Feature']]
y_test2 = pd.read_csv('data/y_test2.csv')

In [17]:
Grid_Search_Results = pd.DataFrame(columns = ["Number of Trees", "Learning Rate (LR)", 'Subsample', '% Features', 'Weight of Default',"AUC Train", "AUC Test 1", "AUC Test 2"])

Counter = 0
for n_trees in [300]: # 50, 100, 300 respectively due to memory constraints
    for lr in [0.01, 0.1]:
        for subsample in [0.5, 0.8]:
            for colsample in [0.5, 1.0]:
                for weight in [1, 5, 10]:
                    xgb_instance = xgb.XGBClassifier(n_estimators= n_trees, learning_rate = lr,subsample=subsample, colsample_bytree=colsample, scale_pos_weight=weight)                    
                    model = xgb_instance.fit(X_train, y_train)
                    
                    Grid_Search_Results.loc[Counter,"Number of Trees"] = n_trees
                    Grid_Search_Results.loc[Counter,"Learning Rate (LR)"] = lr
                    Grid_Search_Results.loc[Counter,"Subsample"] = subsample
                    Grid_Search_Results.loc[Counter,"% Features"] = colsample
                    Grid_Search_Results.loc[Counter,"Weight of Default"] = weight
                    
                    Grid_Search_Results.loc[Counter,"AUC Train"] = roc_auc_score(y_train, model.predict_proba(X_train)[:,1])
                    Grid_Search_Results.loc[Counter,"AUC Test 1"] = roc_auc_score(y_test1, model.predict_proba(X_test1)[:,1])
                    Grid_Search_Results.loc[Counter,"AUC Test 2"] = roc_auc_score(y_test2, model.predict_proba(X_test2)[:,1])

                    Counter = Counter + 1
                    print('complete iteration:', Counter)
print('Grid search completed')
Grid_Search_Results.to_csv("output/Grid_Search_Results3.csv")


complete iteration: 1
complete iteration: 2
complete iteration: 3
complete iteration: 4
complete iteration: 5
complete iteration: 6
complete iteration: 7
complete iteration: 8
complete iteration: 9
complete iteration: 10
complete iteration: 11
complete iteration: 12
complete iteration: 13
complete iteration: 14
complete iteration: 15
complete iteration: 16
complete iteration: 17
complete iteration: 18
complete iteration: 19
complete iteration: 20
complete iteration: 21
complete iteration: 22
complete iteration: 23
complete iteration: 24
Grid search completed


In [26]:
# Grid search results analysis
# List of file paths
file_paths = [f"output/Grid_Search_Results{i}.csv" for i in range(1, 4)]

# Read and concatenate all CSV files
Grid_Search_Results = pd.concat([pd.read_csv(file) for file in file_paths], ignore_index=True)

# Display the concatenated DataFrame
print(Grid_Search_Results)


    Number of Trees  Learning Rate (LR)  Subsample  % Features  \
0                50                0.01        0.5         0.5   
1                50                0.01        0.5         0.5   
2                50                0.01        0.5         0.5   
3                50                0.01        0.5         1.0   
4                50                0.01        0.5         1.0   
..              ...                 ...        ...         ...   
67              300                0.10        0.8         0.5   
68              300                0.10        0.8         0.5   
69              300                0.10        0.8         1.0   
70              300                0.10        0.8         1.0   
71              300                0.10        0.8         1.0   

    Weight of Default  AUC Train  AUC Test 1  AUC Test 2  Unnamed: 0  
0                   1   0.927436    0.927852    0.927001         NaN  
1                   5   0.925855    0.926490    0.925382         

In [30]:
best_model = Grid_Search_Results[['AUC Train', 'AUC Test 1', 'AUC Test 2']].mean(axis=1).idxmax()
best_parameters = Grid_Search_Results.iloc[best_model, :5].to_dict()

In [31]:
print(best_parameters)
best_model

{'Number of Trees': 300.0, 'Learning Rate (LR)': 0.1, 'Subsample': 0.8, '% Features': 1.0, 'Weight of Default': 1.0}


69

In [33]:
# Train the best model
best_xgb_model = xgb.XGBClassifier(
    n_estimators=int(best_parameters['Number of Trees']),
    learning_rate=best_parameters['Learning Rate (LR)'],
    subsample=best_parameters['Subsample'],
    colsample_bytree=best_parameters['% Features'],
    scale_pos_weight=best_parameters['Weight of Default']
)
best_xgb_model.fit(X_train, y_train)

# Save the best model
best_xgb_model.save_model('best_xgb_model.json')

In [39]:
# Load the model
loaded_model = xgb.XGBClassifier()
loaded_model.load_model('output/best_xgb_model.json')

In [43]:
# step 10: Neural Network
# Data Preprocessing
outliers = pd.DataFrame(columns = ['Feature', 'p1', 'p99'])

counter = 1
for col in X_train.columns:
    p1 = X_train[col].quantile(0.01)
    p99 = X_train[col].quantile(0.99)
    outliers.loc[counter] = [col, p1, p99]
    counter += 1
    print('complete iteration:', counter)
print('----Calculation Complete----')
outliers.to_csv('output/outliers.csv', index=False)



complete iteration: 2
complete iteration: 3
complete iteration: 4
complete iteration: 5
complete iteration: 6
complete iteration: 7
complete iteration: 8
complete iteration: 9
complete iteration: 10
complete iteration: 11
complete iteration: 12
complete iteration: 13
complete iteration: 14
complete iteration: 15
complete iteration: 16
complete iteration: 17
complete iteration: 18
complete iteration: 19
complete iteration: 20
complete iteration: 21
complete iteration: 22
complete iteration: 23
complete iteration: 24
complete iteration: 25
complete iteration: 26
complete iteration: 27
complete iteration: 28
complete iteration: 29
complete iteration: 30
complete iteration: 31
complete iteration: 32
complete iteration: 33
complete iteration: 34
----Calculation Complete----


In [ ]:
# replace outliers with p1 and p99
X_train = X_train.clip(lower = X_train.quantile(0.01), upper = X_train.quantile(0.99), axis = 1)
X_test1 = X_test1.clip(lower = X_train.quantile(0.01), upper = X_train.quantile(0.99), axis = 1)
X_test2 = X_test2.clip(lower = X_train.quantile(0.01), upper = X_train.quantile(0.99), axis = 1)

In [61]:
# feature scaling
scaler = StandardScaler()
scaler.fit(X_train)

X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test1_scaled = pd.DataFrame(scaler.transform(X_test1), columns = X_train.columns)
X_test2_scaled = pd.DataFrame(scaler.transform(X_test2), columns = X_train.columns)


In [66]:
# fill missing values with 0
print('Number of missing values in X_train:', X_train_scaled.isnull().sum().sum())
print('Number of missing values in X_test1:', X_test1_scaled.isnull().sum().sum())
print('Number of missing values in X_test2:', X_test2_scaled.isnull().sum().sum())


Number of missing values in X_train: 2898599
Number of missing values in X_test1: 621000
Number of missing values in X_test2: 621458


In [68]:
X_train_scaled.fillna(0, inplace=True)
X_test1_scaled.fillna(0, inplace=True)
X_test2_scaled.fillna(0, inplace=True)

In [69]:
# Check the number of missing values again
print('Number of missing values in X_train:', X_train_scaled.isnull().sum().sum())
print('Number of missing values in X_test1:', X_test1_scaled.isnull().sum().sum())
print('Number of missing values in X_test2:', X_test2_scaled.isnull().sum().sum())

Number of missing values in X_train: 0
Number of missing values in X_test1: 0
Number of missing values in X_test2: 0


In [70]:
X_train_scaled

,P_2,B_7,B_38_2.0,D_44,D_42,B_9,D_45,B_22,S_3,B_3,...,B_4,D_79,D_46,D_39,B_10,B_5,S_23,B_6,B_11,B_2
0,-0.374206,3.078623,-0.737201,0.065124,0.000000,1.551046,-0.802174,1.770830,-0.000216,2.051342,...,1.142222,-0.307905,-0.223151,-0.444705,-0.066297,-0.166801,-0.054172,-0.131853,2.169141,-1.491802
1,-0.436828,0.972400,-0.737201,4.019297,0.000000,1.253284,2.072252,15.198294,-0.381406,1.140559,...,2.409860,-0.283741,0.097797,-0.208156,-0.060158,-0.085484,-0.048562,-0.122735,2.605975,-1.465750
2,1.143916,-0.657959,1.356482,-0.508500,0.000000,-0.625478,4.700128,-0.456450,-0.580914,-0.527109,...,-0.523050,-0.308897,-0.474168,-0.555178,0.021238,-0.043669,-0.047146,0.053462,-0.352307,0.943097
3,-0.474731,-0.769824,1.356482,-0.521303,-0.494602,-0.613165,-0.953270,-0.467049,1.101270,-0.531135,...,-0.742834,-0.295964,0.000000,-0.546379,0.044028,-0.100438,-0.049544,0.169204,-0.497060,0.962879
4,0.992171,-0.683196,-0.737201,-0.531804,0.000000,-0.629450,0.960398,-0.436698,-0.159567,-0.525143,...,-0.569517,-0.313490,-0.723551,-0.534440,0.020547,-0.205472,-0.051005,0.013287,-0.486882,0.480384
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
774952,0.002308,-0.569588,-0.737201,-0.515162,0.000000,-0.126457,1.144739,-0.464439,-0.623059,0.287494,...,0.001642,-0.304841,0.791996,1.527999,-0.033775,0.934618,-0.048433,-0.049420,-0.426991,-0.686471
774953,-1.177257,0.894441,-0.737201,3.459608,-0.593183,1.432379,-0.939099,1.759826,1.494743,0.631356,...,2.126663,-0.301010,1.374986,-0.541627,-0.062492,-0.193591,-0.048150,-0.119691,0.259954,-1.484008
774954,-2.190122,1.518246,-0.737201,0.037062,0.000000,1.600726,0.986997,1.802060,2.540128,1.425908,...,0.632863,-0.277882,-0.068328,0.762514,-0.062305,-0.127641,-0.050036,-0.125553,0.674297,-1.362300
774955,0.399516,-0.766275,1.356482,-0.491500,0.000000,-0.563742,-0.097036,-0.447427,-0.881684,-0.556015,...,-0.741363,-0.296087,0.000000,0.314135,0.020889,1.659218,-0.048989,0.065434,-0.504940,0.942143


In [71]:
X_train_scaled.to_csv('data/X_train_scaled.csv', index=False)
X_test1_scaled.to_csv('data/X_test1_scaled.csv', index=False)
X_test2_scaled.to_csv('data/X_test2_scaled.csv', index=False)

In [ ]:
# build neural network